In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [7]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter, read_saved_epochs

from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry_new, decoder_with_fc_only
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
n_pc_samples = 2048
do_training = True
first_time_running = True
load_model = False
seed = 42
max_training_epochs = 2000
loss = 'chamfer'
z_rotate = True
experiment_name = '2_all_snc_rotated_mlp_arch_with_bnorm_no_dropout' + str(n_pc_samples) +  'pts_' + loss

In [5]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
full_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/all_snc', experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/2_all_snc_rotated_mlp_arch_with_bnorm_no_dropout2048pts_chamfer'

In [6]:
n_threads = 40
verbose = True
full_file_names = [f for f in files_in_subdirs(full_pclouds_path, '.ply')]
pclouds, model_names, syn_ids = load_point_clouds_from_filenames(full_file_names, n_threads, snc_loader, verbose)
model_unames = model_names + '.' + syn_ids
train_data = PointCloudDataSet(pclouds, labels=model_unames)

57449 pclouds were loaded. They belong in 57 shape-classes.


/orions4-zfs/projects/optas/Git_Repos/tf_lab/point_clouds/in_out.py:38: UserWarning: Point clouds with the same model name were loaded.
  warnings.warn('Point clouds with the same model name were loaded.')


In [9]:
n_input = [n_pc_samples, 3]

encoder_args = {'n_filters': [128, 128, 256, 512],
                'filter_sizes': [1, 1, 1, 1],
                'strides': [1, 1, 1, 1],
                'b_norm': True
               }

decoder_args = {'layer_sizes': [1024, 2048, np.prod(n_input)],
                'b_norm': True
               }

if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    
else:    
    conf = Conf(
                n_input = n_input,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                denoising = False,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 10,
                learning_rate = 0.0005,
                z_rotate = z_rotate,
                encoder = encoder_with_convs_and_symmetry_new,
                encoder_args = encoder_args,
                decoder = decoder_with_fc_only,
                decoder_args = decoder_args,            
               )
    conf.allow_gpu_growth = False
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))

In [10]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    last_epoch = saved_epochs[-1]
    ae.restore_model(conf.train_dir, last_epoch, verbose=True)

In [11]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '1.9179', 'loss=', '0.011056859')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/2_all_snc_rotated_mlp_arch_with_bnorm_no_dropout2048pts_chamfer/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/2_all_snc_rotated_mlp_arch_with_bnorm_no_dropout2048pts_chamfer/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0002', 'training time (minutes)=', '1.9263', 'loss=', '0.003144248')
('Epoch:', '0003', 'training time (minutes)=', '1.9559', 'loss=', '0.002146730')
('Epoch:', '0004', 'training time (minutes)=', '2.1156', 'loss=', '0.001834057')
('Epoch:', '0005', 'training time (minutes)=', '2.0769', 'loss=', '0.001639452')
('Epoch:', '0006', 'training time (minutes)=', '2.0201', 'loss=', '0.001392193')
('Epoch:', '0007', 'training time (minutes)=', '1.9808', 'loss=', '0.001332284')
('Epoch:', '0008', 'training time (minutes)=', '1.9750', 'loss=', '0.001298236')
('Epoch:', '0009', 'training time (minutes)=', '1.9526', 'loss=', '0.001122028')
('Epoch:', '0010', 'training time (minutes)=', '2.0199', 'loss=', '0.001064924')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/2_all_snc_rotated_mlp_arch_with_bnorm_no_dropout2048pts_chamfer/models.ckpt-10 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/2_all_snc_rotated_mlp_arch_with_bnorm_no_dropout2048pts_chamfer/models.ckpt-10 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0011', 'training time (minutes)=', '1.9527', 'loss=', '0.001022843')
('Epoch:', '0012', 'training time (minutes)=', '1.9218', 'loss=', '0.000993127')
('Epoch:', '0013', 'training time (minutes)=', '1.9317', 'loss=', '0.000949123')
('Epoch:', '0014', 'training time (minutes)=', '1.9587', 'loss=', '0.000941023')
('Epoch:', '0015', 'training time (minutes)=', '1.9547', 'loss=', '0.000936503')
('Epoch:', '0016', 'training time (minutes)=', '1.9937', 'loss=', '0.000870901')
('Epoch:', '0017', 'training time (minutes)=', '1.9332', 'loss=', '0.000864837')
('Epoch:', '0018', 'training time (minutes)=', '1.9542', 'loss=', '0.000832542')
('Epoch:', '0019', 'training time (minutes)=', '1.9861', 'loss=', '0.000837028')
('Epoch:', '0020', 'training time (minutes)=', '1.9442', 'loss=', '0.000819186')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/2_all_snc_rotated_mlp_arch_with_bnorm_no_dropout2048pts_chamfer/models.ckpt-20 is not in all_model_checkpoint_path

...

INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/2_all_snc_rotated_mlp_arch_with_bnorm_no_dropout2048pts_chamfer/models.ckpt-680 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0681', 'training time (minutes)=', '1.9472', 'loss=', '0.000402760')
('Epoch:', '0682', 'training time (minutes)=', '1.9516', 'loss=', '0.000402899')
('Epoch:', '0683', 'training time (minutes)=', '1.9852', 'loss=', '0.000397168')
('Epoch:', '0684', 'training time (minutes)=', '1.9593', 'loss=', '0.000396872')
('Epoch:', '0685', 'training time (minutes)=', '1.9709', 'loss=', '0.000403238')
('Epoch:', '0686', 'training time (minutes)=', '1.9628', 'loss=', '0.000398128')
('Epoch:', '0687', 'training time (minutes)=', '1.9238', 'loss=', '0.000404441')
('Epoch:', '0688', 'training time (minutes)=', '1.9225', 'loss=', '0.000399523')
('Epoch:', '0689', 'training time (minutes)=', '1.9433', 'loss=', '0.000400998')
('Epoch:', '0690', 'training time (minutes)=', '1.9926', 'loss=', '0.000404087')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/2_all_snc_rotated_mlp_arch_with_bnorm_no_dropout2048pts_chamfer/models.ckpt-690 is not in all_model_checkpoint_pat

INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/2_all_snc_rotated_mlp_arch_with_bnorm_no_dropout2048pts_chamfer/models.ckpt-690 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0691', 'training time (minutes)=', '1.9256', 'loss=', '0.000397897')
('Epoch:', '0692', 'training time (minutes)=', '1.9310', 'loss=', '0.000407405')
('Epoch:', '0693', 'training time (minutes)=', '1.9331', 'loss=', '0.000405973')


KeyboardInterrupt: 